# Step 3B: Model Scoring evaluation

Using the results data set constructed in the `./notebooks/3b_model_scoring` Jupyter notebook, this notebook loads the data scores the observations. 

**Note:** This notebook will take about 1 minutes to execute all cells, depending on the compute configuration you have setup.

In [2]:
# import the libraries

# For some data handling
import numpy as np
from pyspark.ml import PipelineModel
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer


# The scoring uses the same feature engineering script used to train the model
results_table = 'results_output'

In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text("results_data", results_table)

In [4]:

# make predictions. The Pipeline does all the same operations on the test data
predictions =  spark.sql("SELECT * FROM " + dbutils.widgets.get("results_data"))

# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas()
confuse.head()

Out[ 9 ]: 
 indexedLabel_prediction 0.0 1.0 2.0 3.0 4.0
0 0.0 89577 80 33 39 11
1 1.0 1702 593 0 2 1
2 2.0 1198 3 399 0 1
3 3.0 930 2 0 412 1
4 4.0 787 4 0 2 268

The confusion matrix lists each true component failure in rows and the predicted value in columns. Labels numbered 0.0 corresponds to no component failures. Labels numbered 1.0 through 4.0 correspond to failures in one of the four components in the machine. As an example, the third number in the top row indicates how many days we predicted component 2 would fail, when no components actually did fail. The second number in the second row, indicates how many days we correctly predicted a component 1 failure within the next 7 days.

We read the confusion matrix numbers along the diagonal as correctly classifying the component failures. Numbers above the diagonal indicate the model incorrectly predicting a failure when non occured, and those below indicate incorrectly predicting a non-failure for the row indicated component failure.

When evaluating classification models, it is convenient to reduce the results in the confusion matrix into a single performance statistic. However, depending on the problem space, it is impossible to always use the same statistic in this evaluation. Below, we calculate four such statistics.

- **Accuracy**: reports how often we correctly predicted the labeled data. Unfortunatly, when there is a class imbalance (a large number of one of the labels relative to others), this measure is biased towards the largest class. In this case non-failure days.

Because of the class imbalance inherint in predictive maintenance problems, it is better to look at the remaining statistics instead. Here positive predictions indicate a failure.

- **Precision**: Precision is a measure of how well the model classifies the truely positive samples. Precision depends on falsely classifying negative days as positive.

- **Recall**: Recall is a measure of how well the model can find the positive samples. Recall depends on falsely classifying positive days as negative.

- **F1**: F1 considers both the precision and the recall. F1 score is the harmonic average of precision and recall. An F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0.

These metrics make the most sense for binary classifiers, though they are still useful for comparision in our multiclass setting. Below we calculate these evaluation statistics for the selected classifier, and post them back to the AML workbench run time page for tracking between experiments.

In [6]:
# select (prediction, true label) and compute test error
# select (prediction, true label) and compute test error
# True positives - diagonal failure terms 
tp = confuse['1.0'][1]+confuse['2.0'][2]+confuse['3.0'][3]+confuse['4.0'][4]

# False positves - All failure terms - True positives
fp = np.sum(np.sum(confuse[['1.0', '2.0','3.0','4.0']])) - tp

# True negatives 
tn = confuse['0.0'][0]

# False negatives total of non-failure column - TN
fn = np.sum(np.sum(confuse[['0.0']])) - tn

# Accuracy is diagonal/total 
acc_n = tn + tp
acc_d = np.sum(np.sum(confuse[['0.0','1.0', '2.0','3.0','4.0']]))
acc = acc_n/acc_d

# Calculate precision and recall.
prec = tp/(tp+fp)
rec = tp/(tp+fn)

# Print the evaluation metrics to the notebook
print("Accuracy = %g" % acc)
print("Precision = %g" % prec)
print("Recall = %g" % rec )
print("F1 = %g" % (2.0 * prec * rec/(prec + rec)))
print("")

Accuracy = 0.950065
Precision = 0.903296
Recall = 0.265861
F1 = 0.410811

Remember that this is a simulated data set. We would expect a model built on real world data to behave very differently. The accuracy may still be close to one, but the precision and recall numbers would be much lower.

In [8]:
predictions.toPandas().head(20)

Out[ 11 ]: 
 machineID dt_truncated label_e \
0 45 2016-01-01 12:00:00 0.0 
1 45 2016-01-01 00:00:00 0.0 
2 45 2015-12-31 12:00:00 0.0 
3 45 2015-12-31 00:00:00 0.0 
4 45 2015-12-30 12:00:00 0.0 
5 45 2015-12-30 00:00:00 0.0 
6 45 2015-12-29 12:00:00 0.0 
7 45 2015-12-29 00:00:00 0.0 
8 45 2015-12-28 12:00:00 0.0 
9 45 2015-12-28 00:00:00 0.0 
10 45 2015-12-27 12:00:00 0.0 
11 45 2015-12-27 00:00:00 0.0 
12 45 2015-12-26 12:00:00 0.0 
13 45 2015-12-26 00:00:00 0.0 
14 45 2015-12-25 12:00:00 0.0 
15 45 2015-12-25 00:00:00 0.0 
16 45 2015-12-24 12:00:00 0.0 
17 45 2015-12-24 00:00:00 0.0 
18 45 2015-12-23 12:00:00 0.0 
19 45 2015-12-23 00:00:00 0.0 

 features indexedLabel \
0 (180.192497527, 482.320763677, 123.340058678, ... 0.0 
1 [185.441169349, 470.689865796, 115.674212621, ... 0.0 
2 [184.01031328, 444.755575817, 103.049769247, 4... 0.0 
3 [174.139126307, 461.528239244, 99.4343354818, ... 0.0 
4 [169.351114918, 444.253012934, 96.7993394654, ... 0.0 
5 [170.275399706, 463.208150561, 100.510935031, ... 0.0 
6 [175.87402359, 443.653284824, 102.74427056, 41... 0.0 
7 [170.26204021, 456.053290902, 99.2346402263, 3... 0.0 
8 [169.69033341, 450.097554637, 98.3041556692, 4... 0.0 
9 [169.857394657, 450.76328442, 100.601895961, 3... 0.0 
10 [171.283378445, 463.236632798, 100.953781999, ... 0.0 
11 [166.790548049, 456.412326239, 101.412157477, ... 0.0 
12 [171.41636184, 448.722025312, 102.6334959, 41.... 0.0 
13 [173.458704761, 439.53941217, 100.886214922, 3... 0.0 
14 [170.397340734, 423.636355064, 98.3771369872, ... 0.0 
15 [173.591162765, 457.849396598, 99.9297326911, ... 0.0 
16 [170.345907413, 443.917666998, 100.225179762, ... 0.0 
17 [170.220003322, 449.074548101, 100.933044339, ... 0.0 
18 [176.348007602, 462.315166576, 101.185218034, ... 0.0 
19 [172.05879022, 461.15781296, 99.5618742677, 39... 0.0 

 indexedFeatures \
0 (180.192497527, 482.320763677, 123.340058678, ... 
1 [185.441169349, 470.689865796, 115.674212621, ... 
2 [184.01031328, 444.755575817, 103.049769247, 4... 
3 [174.139126307, 461.528239244, 99.4343354818, ... 
4 [169.351114918, 444.253012934, 96.7993394654, ... 
5 [170.275399706, 463.208150561, 100.510935031, ... 
6 [175.87402359, 443.653284824, 102.74427056, 41... 
7 [170.26204021, 456.053290902, 99.2346402263, 3... 
8 [169.69033341, 450.097554637, 98.3041556692, 4... 
9 [169.857394657, 450.76328442, 100.601895961, 3... 
10 [171.283378445, 463.236632798, 100.953781999, ... 
11 [166.790548049, 456.412326239, 101.412157477, ... 
12 [171.41636184, 448.722025312, 102.6334959, 41.... 
13 [173.458704761, 439.53941217, 100.886214922, 3... 
14 [170.397340734, 423.636355064, 98.3771369872, ... 
15 [173.591162765, 457.849396598, 99.9297326911, ... 
16 [170.345907413, 443.917666998, 100.225179762, ... 
17 [170.220003322, 449.074548101, 100.933044339, ... 
18 [176.348007602, 462.315166576, 101.185218034, ... 
19 [172.05879022, 461.15781296, 99.5618742677, 39... 

 rawPrediction \
0 [118.785713683, 3.48758406511, 35.7940710937, ... 
1 [116.370908033, 2.05892476743, 46.3216448581, ... 
2 [171.447680013, 3.33770267638, 22.7224065373, ... 
3 [193.008138511, 2.17063613306, 1.60241779063, ... 
4 [193.099042951, 2.13327639533, 1.58032663083, ... 
5 [193.066949671, 2.136873084, 1.5884307985, 2.4... 
6 [192.662238374, 2.27853237488, 1.75396845137, ... 
7 [193.057677349, 2.13423503345, 1.58097464703, ... 
8 [193.071451893, 2.15498764926, 1.56086496971, ... 
9 [193.020327502, 2.15434049402, 1.58058369662, ... 
10 [193.097884317, 2.12985470763, 1.5817092509, 2... 
11 [193.070147433, 2.1456210149, 1.57852482579, 2... 
12 [193.036583273, 2.19186076134, 1.58872817252, ... 
13 [192.901278737, 2.40861110865, 1.54780552487, ... 
14 [182.44733574, 11.7418047753, 1.43966552651, 2... 
15 [193.049306044, 2.15880993923, 1.58841225442, ... 
16 [192.203478203, 2.07580050185, 2.80643854405, ... 
17 [193.031468245, 2.15033023846, 1.60139002262, ... 
18 [192.606831097, 2.28657633497, 1.83542996985, ... 
19 [193.002154147, 2.158830313, 1.6379268739

In [9]:
print(predictions.summary())

DataFrame[summary: string, machineID: string, label_e: string, indexedLabel: string, prediction: string]

In [10]:
predictions.explain()

== Physical Plan ==
*(1) FileScan parquet default.results_output[machineID#78069L,dt_truncated#78070,label_e#78071,features#78072,indexedLabel#78073,indexedFeatures#78074,rawPrediction#78075,probability#78076,prediction#78077] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/results_output], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<machineID:bigint,dt_truncated:timestamp,label_e:double,features:struct<type:tinyint,size:i...

# Conclusion

This concludes this scenario. You can modify these notebooks to customize your own use case solution.